In [2]:
!pip install datasets  # Huggingface datasets package
!pip install google-cloud
!pip install google-cloud-run
!pip install google-cloud-logging
!pip install multiprocess  # For bulk evaluation
!pip install radon  # More code complexity metrics
!pip3 install datasets  # Huggingface datasets package
!pip3 install google-cloud
!pip3 install google-cloud-run
!pip3 install google-cloud-logging
!pip3 install multiprocess  # For bulk evaluation
!pip3 install radon  # More code complexity metrics


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip

[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip avail

In [4]:
from eval import DATA, evaluate, bulk_evaluate

mbpp = DATA["mbpp"]  # train, validation, and test
humaneval = DATA["openai_humaneval"]  # test only

In [5]:
# INDIVIDUAL TEST
canonical_solution = humaneval["test"][0]["prompt"] + humaneval["test"][0]["canonical_solution"]
print(type(canonical_solution))
evaluate(
    dataset="openai_humaneval",
    split="test",
    task_id=0,
    code=canonical_solution  # or alternatively YOUR_CODE_HERE
)

canonical_solution = mbpp["test"][0]["code"]
evaluate(
    dataset="mbpp",
    split="test",
    task_id=0,
    code=canonical_solution  # or alternatively YOUR_CODE_HERE
)

<class 'str'>
Executing...
Executing...


{'dataset': 'mbpp',
 'split': 'test',
 'task_id': 0,
 'result': 'passed',
 'compiled': True,
 'passed_tests': True,
 'avg_test_time': 6.313653333336333e-05,
 'loc': 10,
 'lloc': 12,
 'sloc': 10,
 'comments': 0,
 'multi': 0,
 'blank': 0,
 'single_comments': 0,
 'CC': 5,
 'h1': 4,
 'h2': 10,
 'N1': 9,
 'N2': 16,
 'vocabulary': 14,
 'length': 25,
 'calculated_length': 41.219280948873624,
 'volume': 95.18387305144009,
 'difficulty': 3.2,
 'effort': 304.58839376460827,
 'time': 16.921577431367126,
 'bugs': 0.03172795768381336,
 'MI': 61.93233773944609}

In [8]:
import csv

def extract_after_substring(full_string, substring):
    index = full_string.find(substring)
    if index != -1:
        return full_string[index + len(substring):]
    else:
        return ""

def extract_between_markers(full_string, start_marker, end_marker):
    start_index = full_string.find(start_marker)
    end_index = full_string.find(end_marker, start_index + len(start_marker))
    if start_index != -1 and end_index != -1:
        return full_string[start_index + len(start_marker):end_index].strip()
    else:
        return ""

def extract_python_code(full_string):
    extracted_string = extract_after_substring(full_string, "### Response")
    return extract_between_markers(extracted_string, "```python", "```")


In [10]:
def parsed_csv(file_path, results):
  keys = ['error', 'dataset', 'split', 'task_id', 'result', 'avg_test_time', 'passed_tests', 'compiled', 'loc', 'lloc', 'sloc', 'comments', 'multi', 'blank', 'single_comments', 'CC', 'h1', 'h2', 'N1', 'N2', 'vocabulary', 'length', 'calculated_length', 'volume', 'difficulty', 'effort', 'time', 'bugs', 'MI']
       
  with open(file_path, 'w', newline='') as csvfile:
      writer = csv.DictWriter(csvfile, fieldnames=keys)
      writer.writeheader()
      for result in results:
          # print(result)
          if type(result) == str:
            writer.writerow({"error": result})
          else:
            writer.writerow(result)

In [11]:
csv_file_path = 'humaneval_rules_refactored.csv'
humaneval_output_values = []
with open(csv_file_path, 'r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        humaneval_output_values.append(row['output'])

humaneval_snippets=[]
for i in humaneval_output_values:
  humaneval_snippets.append(extract_after_substring(i, "### Response"))
  
humaneval_bulk_output = bulk_evaluate(
    dataset="openai_humaneval",
    split="test",
    code=humaneval_snippets, # one for each task in HumanEval test
)


parsed_csv('./humaneval_metrics.csv', humaneval_bulk_output)

Sending 164 jobs for execution...
164 jobs sent
Fetching 164 job results...
10/164 fetched
20/164 fetched
30/164 fetched
40/164 fetched
50/164 fetched
60/164 fetched
70/164 fetched
80/164 fetched
90/164 fetched
100/164 fetched
110/164 fetched
120/164 fetched
130/164 fetched
140/164 fetched
150/164 fetched
160/164 fetched
164 job results fetched


In [12]:
csv_file_path = 'mbpp_rules_refactored.csv'
mbpp_output_values = []
with open(csv_file_path, 'r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        mbpp_output_values.append(row['output'])

mbpp_snippets=[]
for i in mbpp_output_values:
  mbpp_snippets.append(extract_after_substring(i, "### Response"))
  
mbpp_bulk_output = bulk_evaluate(
    dataset="mbpp",
    split="test",
    code=mbpp_snippets,
)

parsed_csv('./mbpp_metrics.csv', mbpp_bulk_output)


Sending 500 jobs for execution...
500 jobs sent
Fetching 500 job results...
10/500 fetched
20/500 fetched
30/500 fetched
40/500 fetched
50/500 fetched
60/500 fetched
70/500 fetched
80/500 fetched
90/500 fetched
100/500 fetched
110/500 fetched
120/500 fetched
130/500 fetched
140/500 fetched
150/500 fetched
160/500 fetched
170/500 fetched
180/500 fetched
190/500 fetched
200/500 fetched
210/500 fetched
220/500 fetched
230/500 fetched
240/500 fetched
250/500 fetched
260/500 fetched
270/500 fetched
280/500 fetched
290/500 fetched
300/500 fetched
310/500 fetched
320/500 fetched
330/500 fetched
340/500 fetched
350/500 fetched
360/500 fetched
370/500 fetched
380/500 fetched
390/500 fetched
400/500 fetched
410/500 fetched
420/500 fetched
430/500 fetched
440/500 fetched
450/500 fetched
460/500 fetched
470/500 fetched
480/500 fetched
490/500 fetched
500 job results fetched
